In [7]:
import csv
import numpy as np

In [8]:
# Define object for data to be analyzed
class Data:
    def __init__(self, step, r_embb, g_embb, u_embb, r_urllc, g_urllc, u_urllc, r_miot, g_miot, u_miot, r_mmtc, g_mmtc, u_mmtc, r_voice, g_voice, u_voice, band_remaining):
        self.step = step
        self.r_embb = r_embb
        self.g_embb = g_embb
        self.u_embb = u_embb
        self.r_urllc = r_urllc
        self.g_urllc = g_urllc
        self.u_urllc = u_urllc
        self.r_miot = r_miot
        self.g_miot = g_miot
        self.u_miot = u_miot
        self.r_mmtc = r_mmtc
        self.g_mmtc = g_mmtc
        self.u_mmtc = u_mmtc
        self.r_voice = r_voice
        self.g_voice = g_voice
        self.u_voice = u_voice
        self.band_remaining = band_remaining

# Define object for finale distribution per round
class TrueDistribution:
    def __init__(self, time, true_embb, true_urllc, true_miot, true_mmtc, true_voice, band):
        self.time = time
        self.true_embb = true_embb
        self.true_urllc = true_urllc
        self.true_miot = true_miot
        self.true_mmtc = true_mmtc
        self.true_voice = true_voice
        self.band = band
class MakeCsv:
    def __init__(self, step, r_embb, t_embb, r_urllc, t_urllc, r_miot, t_miot, r_mmtc, t_mmtc, r_voice, t_voice, band): 
        self.step = step
        self.r_embb = r_embb
        
        self.t_embb = t_embb
        self.r_urllc = r_urllc
        
        self.t_urllc = t_urllc
        self.r_miot = r_miot
        
        self.t_miot = t_miot
        self.r_mmtc = r_mmtc
        
        self.t_mmtc = t_mmtc
        self.r_voice = r_voice
        
        self.t_voice = t_voice
        self.band = band 

In [9]:
# Parameters and functions
bs_band = 20000000000
embb_fixed = 0.55
urllc_fixed = 0.05
miot_fixed  = 0.15
mmtc_fixed = 0.15
voice_fixed = 0.1

def get_ratios_br(band_available):
    ratio_embb = int(embb_fixed * band_available)
    ratio_urllc = int(urllc_fixed * band_available)
    ratio_miot = int(miot_fixed * band_available)
    ratio_mmtc = int(mmtc_fixed * band_available)
    ratio_voice = int(voice_fixed * band_available)
        
    #TODO: better way to distributed band when some goes to 0
        
    ratios  = {
        'URLLC': ratio_urllc,
        'eMBB': ratio_embb,
        'mMTC': ratio_mmtc,
        'voice': ratio_voice,
        'MIoT': ratio_miot
    }
    
    return ratios


In [10]:
# Read data and put them in a list to easy manage
with open('input.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    i = 0
    data_list = []
    
    for row in csv_reader:
        embb_g =int(row['embb_rate'])
        embb_u = int(row['embb_users'])
        embb_r = int(row['embb_req'])
        urllc_g = int(row['urllc_rate'])
        urllc_u = int(row['urllc_users'])
        urllc_r = int(row['urllc_req'])
        miot_g = int(row['miot_rate'])
        miot_u = int(row['miot_users'])
        miot_r = int(row['miot_req'])
        mmtc_g = int(row['mmtc_rate'])
        mmtc_u = int(row['mmtc_users'])
        mmtc_r = int(row['mmtc_req']) 
        voice_g = int(row['voice_rate'])
        voice_u = int(row['voice_users'])
        voice_r = int(row['voice_req'])
        band_rem = bs_band - (embb_g + urllc_g + miot_g + mmtc_g + voice_g)
        
        data_step = Data(i, embb_r, embb_g, embb_u, urllc_r, urllc_g, urllc_u, miot_r, miot_g, miot_u, mmtc_r, mmtc_g, mmtc_u, voice_r, voice_g, voice_u, band_rem)
        i += 1
        
        data_list.append(data_step)
        

In [11]:
# Compute new distribution
i = 0
true_ratios = []

for item in data_list:
    
    print('STEP', i)
    
    br = item.band_remaining
    total_req = item.r_embb  + item.r_urllc + item.r_miot + item.r_mmtc + item.r_voice
    embb_g_temp, urllc_g_temp, miot_g_temp, mmtc_g_temp, voice_g_temp = item.g_embb, item.g_urllc, item.g_miot, item.g_mmtc, item.g_voice
    new_g_embb, new_g_urllc, new_g_miot, new_g_mmtc, new_g_voice = 0, 0, 0, 0, 0
    
    while br >= 2 and total_req != new_g_embb+new_g_urllc+new_g_miot+new_g_mmtc+new_g_voice:
        available_per_slice = get_ratios_br(br)
        
        if available_per_slice['eMBB'] != 0:
            delta_embb = item.r_embb - embb_g_temp - available_per_slice['eMBB']
            if delta_embb > 0:
                embb_g_temp = embb_g_temp + available_per_slice['eMBB']
                new_g_embb = embb_g_temp
                br = br - available_per_slice['eMBB']
            elif delta_embb == 0:
                new_g_embb = item.r_embb 
                br = br - available_per_slice['eMBB']
            elif delta_embb < 0:
                new_g_embb = item.r_embb
                br = br - available_per_slice['eMBB'] + abs(delta_embb)
        
        if available_per_slice['URLLC'] != 0:
            delta_urllc = item.r_urllc - urllc_g_temp - available_per_slice['URLLC']
            if delta_urllc > 0:
                urllc_g_temp = urllc_g_temp + available_per_slice['URLLC']
                new_g_urllc = urllc_g_temp
                br = br - available_per_slice['URLLC']
            elif delta_urllc == 0:
                new_g_urllc = item.r_urllc
                br = br - available_per_slice['URLLC']
            elif delta_urllc < 0:
                new_g_urllc = item.r_urllc
                br = br - available_per_slice['URLLC'] + abs(delta_urllc)
        
        if available_per_slice['MIoT'] != 0:
            delta_miot = item.r_miot - miot_g_temp - available_per_slice['MIoT']
            if delta_miot > 0:
                miot_g_temp = miot_g_temp + available_per_slice['MIoT']
                new_g_miot = miot_g_temp
                br = br - available_per_slice['MIoT']
            elif delta_miot == 0:
                new_g_miot = item.r_miot
                br = br - available_per_slice['MIoT']
            elif delta_miot < 0:
                new_g_miot = item.r_miot
                br = br - available_per_slice['MIoT'] + abs(delta_miot)
                
        if available_per_slice['mMTC'] != 0:
            delta_mmtc = item.r_mmtc - mmtc_g_temp - available_per_slice['mMTC']
            if delta_mmtc > 0:
                mmtc_g_temp = mmtc_g_temp + available_per_slice['mMTC']
                new_g_mmtc = mmtc_g_temp
                br = br - available_per_slice['mMTC']
            elif delta_mmtc == 0:
                new_g_mmtc = item.r_mmtc
                br = br - available_per_slice['mMTC']
            elif delta_mmtc < 0:
                new_g_mmtc = item.r_mmtc
                br = br - available_per_slice['mMTC'] + abs(delta_mmtc)
        
        if available_per_slice['voice'] != 0:
            delta_voice = item.r_voice - voice_g_temp - available_per_slice['voice']
            if delta_voice > 0:
                voice_g_temp = voice_g_temp + available_per_slice['voice']
                new_g_voice = voice_g_temp
                br = br - available_per_slice['voice']
            elif delta_voice == 0:
                new_g_voice = item.r_voice
                br = br - available_per_slice['voice']
            elif delta_voice < 0:
                new_g_voice = item.r_voice
                br = br - available_per_slice['voice'] + abs(delta_voice)
        if (item.r_embb - new_g_embb) != 0 and (item.r_urllc - new_g_urllc) ==0 and (item.r_miot - new_g_miot) ==0 and (item.r_mmtc - new_g_mmtc)==0 and (item.r_embb - new_g_embb) ==0 and br>0:
            new_g_embb = br
    true = TrueDistribution(i, new_g_embb, new_g_urllc, new_g_miot, new_g_mmtc, new_g_voice, br)
    true_ratios.append(true)
    i += 1   
    

STEP 0
STEP 1
STEP 2
STEP 3
STEP 4
STEP 5
STEP 6
STEP 7
STEP 8
STEP 9
STEP 10
STEP 11
STEP 12
STEP 13
STEP 14
STEP 15
STEP 16
STEP 17
STEP 18
STEP 19
STEP 20
STEP 21
STEP 22
STEP 23
STEP 24
STEP 25
STEP 26
STEP 27
STEP 28
STEP 29
STEP 30
STEP 31
STEP 32
STEP 33
STEP 34
STEP 35
STEP 36
STEP 37
STEP 38
STEP 39
STEP 40
STEP 41
STEP 42
STEP 43
STEP 44
STEP 45
STEP 46
STEP 47
STEP 48
STEP 49
STEP 50
STEP 51
STEP 52
STEP 53
STEP 54
STEP 55
STEP 56
STEP 57
STEP 58
STEP 59
STEP 60
STEP 61
STEP 62
STEP 63
STEP 64
STEP 65
STEP 66
STEP 67
STEP 68
STEP 69
STEP 70
STEP 71
STEP 72
STEP 73
STEP 74
STEP 75
STEP 76
STEP 77
STEP 78
STEP 79
STEP 80
STEP 81
STEP 82
STEP 83
STEP 84
STEP 85
STEP 86
STEP 87
STEP 88
STEP 89
STEP 90
STEP 91
STEP 92
STEP 93
STEP 94
STEP 95
STEP 96
STEP 97
STEP 98
STEP 99
STEP 100
STEP 101
STEP 102
STEP 103
STEP 104
STEP 105
STEP 106
STEP 107
STEP 108
STEP 109
STEP 110
STEP 111
STEP 112
STEP 113
STEP 114
STEP 115
STEP 116
STEP 117
STEP 118
STEP 119
STEP 120
STEP 121
STEP 122
STE

STEP 1390
STEP 1391
STEP 1392
STEP 1393
STEP 1394
STEP 1395
STEP 1396
STEP 1397
STEP 1398
STEP 1399
STEP 1400
STEP 1401
STEP 1402
STEP 1403
STEP 1404
STEP 1405
STEP 1406
STEP 1407
STEP 1408
STEP 1409
STEP 1410
STEP 1411
STEP 1412
STEP 1413
STEP 1414
STEP 1415
STEP 1416
STEP 1417
STEP 1418
STEP 1419
STEP 1420
STEP 1421
STEP 1422
STEP 1423
STEP 1424
STEP 1425
STEP 1426
STEP 1427
STEP 1428
STEP 1429
STEP 1430
STEP 1431
STEP 1432
STEP 1433
STEP 1434
STEP 1435
STEP 1436
STEP 1437
STEP 1438
STEP 1439
STEP 1440
STEP 1441
STEP 1442
STEP 1443
STEP 1444
STEP 1445
STEP 1446
STEP 1447
STEP 1448
STEP 1449
STEP 1450
STEP 1451
STEP 1452
STEP 1453
STEP 1454
STEP 1455
STEP 1456
STEP 1457
STEP 1458
STEP 1459
STEP 1460
STEP 1461
STEP 1462
STEP 1463
STEP 1464
STEP 1465
STEP 1466
STEP 1467
STEP 1468
STEP 1469
STEP 1470
STEP 1471
STEP 1472
STEP 1473
STEP 1474
STEP 1475
STEP 1476
STEP 1477
STEP 1478
STEP 1479
STEP 1480
STEP 1481
STEP 1482
STEP 1483
STEP 1484
STEP 1485
STEP 1486
STEP 1487
STEP 1488
STEP 1489


STEP 2905
STEP 2906
STEP 2907
STEP 2908
STEP 2909
STEP 2910
STEP 2911
STEP 2912
STEP 2913
STEP 2914
STEP 2915
STEP 2916
STEP 2917
STEP 2918
STEP 2919
STEP 2920
STEP 2921
STEP 2922
STEP 2923
STEP 2924
STEP 2925
STEP 2926
STEP 2927
STEP 2928
STEP 2929
STEP 2930
STEP 2931
STEP 2932
STEP 2933
STEP 2934
STEP 2935
STEP 2936
STEP 2937
STEP 2938
STEP 2939
STEP 2940
STEP 2941
STEP 2942
STEP 2943
STEP 2944
STEP 2945
STEP 2946
STEP 2947
STEP 2948
STEP 2949
STEP 2950
STEP 2951
STEP 2952
STEP 2953
STEP 2954
STEP 2955
STEP 2956
STEP 2957
STEP 2958
STEP 2959
STEP 2960
STEP 2961
STEP 2962
STEP 2963
STEP 2964
STEP 2965
STEP 2966
STEP 2967
STEP 2968
STEP 2969
STEP 2970
STEP 2971
STEP 2972
STEP 2973
STEP 2974
STEP 2975
STEP 2976
STEP 2977
STEP 2978
STEP 2979
STEP 2980
STEP 2981
STEP 2982
STEP 2983
STEP 2984
STEP 2985
STEP 2986
STEP 2987
STEP 2988
STEP 2989
STEP 2990
STEP 2991
STEP 2992
STEP 2993
STEP 2994
STEP 2995
STEP 2996
STEP 2997
STEP 2998
STEP 2999
STEP 3000
STEP 3001
STEP 3002
STEP 3003
STEP 3004


STEP 4428
STEP 4429
STEP 4430
STEP 4431
STEP 4432
STEP 4433
STEP 4434
STEP 4435
STEP 4436
STEP 4437
STEP 4438
STEP 4439
STEP 4440
STEP 4441
STEP 4442
STEP 4443
STEP 4444
STEP 4445
STEP 4446
STEP 4447
STEP 4448
STEP 4449
STEP 4450
STEP 4451
STEP 4452
STEP 4453
STEP 4454
STEP 4455
STEP 4456
STEP 4457
STEP 4458
STEP 4459
STEP 4460
STEP 4461
STEP 4462
STEP 4463
STEP 4464
STEP 4465
STEP 4466
STEP 4467
STEP 4468
STEP 4469
STEP 4470
STEP 4471
STEP 4472
STEP 4473
STEP 4474
STEP 4475
STEP 4476
STEP 4477
STEP 4478
STEP 4479
STEP 4480
STEP 4481
STEP 4482
STEP 4483
STEP 4484
STEP 4485
STEP 4486
STEP 4487
STEP 4488
STEP 4489
STEP 4490
STEP 4491
STEP 4492
STEP 4493
STEP 4494
STEP 4495
STEP 4496
STEP 4497
STEP 4498
STEP 4499
STEP 4500
STEP 4501
STEP 4502
STEP 4503
STEP 4504
STEP 4505
STEP 4506
STEP 4507
STEP 4508
STEP 4509
STEP 4510
STEP 4511
STEP 4512
STEP 4513
STEP 4514
STEP 4515
STEP 4516
STEP 4517
STEP 4518
STEP 4519
STEP 4520
STEP 4521
STEP 4522
STEP 4523
STEP 4524
STEP 4525
STEP 4526
STEP 4527


STEP 6005
STEP 6006
STEP 6007
STEP 6008
STEP 6009
STEP 6010
STEP 6011
STEP 6012
STEP 6013
STEP 6014
STEP 6015
STEP 6016
STEP 6017
STEP 6018
STEP 6019
STEP 6020
STEP 6021
STEP 6022
STEP 6023
STEP 6024
STEP 6025
STEP 6026
STEP 6027
STEP 6028
STEP 6029
STEP 6030
STEP 6031
STEP 6032
STEP 6033
STEP 6034
STEP 6035
STEP 6036
STEP 6037
STEP 6038
STEP 6039
STEP 6040
STEP 6041
STEP 6042
STEP 6043
STEP 6044
STEP 6045
STEP 6046
STEP 6047
STEP 6048
STEP 6049
STEP 6050
STEP 6051
STEP 6052
STEP 6053
STEP 6054
STEP 6055
STEP 6056
STEP 6057
STEP 6058
STEP 6059
STEP 6060
STEP 6061
STEP 6062
STEP 6063
STEP 6064
STEP 6065
STEP 6066
STEP 6067
STEP 6068
STEP 6069
STEP 6070
STEP 6071
STEP 6072
STEP 6073
STEP 6074
STEP 6075
STEP 6076
STEP 6077
STEP 6078
STEP 6079
STEP 6080
STEP 6081
STEP 6082
STEP 6083
STEP 6084
STEP 6085
STEP 6086
STEP 6087
STEP 6088
STEP 6089
STEP 6090
STEP 6091
STEP 6092
STEP 6093
STEP 6094
STEP 6095
STEP 6096
STEP 6097
STEP 6098
STEP 6099
STEP 6100
STEP 6101
STEP 6102
STEP 6103
STEP 6104


...

STEP 75809
STEP 75810
STEP 75811
STEP 75812
STEP 75813
STEP 75814
STEP 75815
STEP 75816
STEP 75817
STEP 75818
STEP 75819
STEP 75820
STEP 75821
STEP 75822
STEP 75823
STEP 75824
STEP 75825
STEP 75826
STEP 75827
STEP 75828
STEP 75829
STEP 75830
STEP 75831
STEP 75832
STEP 75833
STEP 75834
STEP 75835
STEP 75836
STEP 75837
STEP 75838
STEP 75839
STEP 75840
STEP 75841
STEP 75842
STEP 75843
STEP 75844
STEP 75845
STEP 75846
STEP 75847
STEP 75848
STEP 75849
STEP 75850
STEP 75851
STEP 75852
STEP 75853
STEP 75854
STEP 75855
STEP 75856
STEP 75857
STEP 75858
STEP 75859
STEP 75860
STEP 75861
STEP 75862
STEP 75863
STEP 75864
STEP 75865
STEP 75866
STEP 75867
STEP 75868
STEP 75869
STEP 75870
STEP 75871
STEP 75872
STEP 75873
STEP 75874
STEP 75875
STEP 75876
STEP 75877
STEP 75878
STEP 75879
STEP 75880
STEP 75881
STEP 75882
STEP 75883
STEP 75884
STEP 75885
STEP 75886
STEP 75887
STEP 75888
STEP 75889
STEP 75890
STEP 75891
STEP 75892
STEP 75893
STEP 75894
STEP 75895
STEP 75896
STEP 75897
STEP 75898
STEP 75899

STEP 76822
STEP 76823
STEP 76824
STEP 76825
STEP 76826
STEP 76827
STEP 76828
STEP 76829
STEP 76830
STEP 76831
STEP 76832
STEP 76833
STEP 76834
STEP 76835
STEP 76836
STEP 76837
STEP 76838
STEP 76839
STEP 76840
STEP 76841
STEP 76842
STEP 76843
STEP 76844
STEP 76845
STEP 76846
STEP 76847
STEP 76848
STEP 76849
STEP 76850
STEP 76851
STEP 76852
STEP 76853
STEP 76854
STEP 76855
STEP 76856
STEP 76857
STEP 76858
STEP 76859
STEP 76860
STEP 76861
STEP 76862
STEP 76863
STEP 76864
STEP 76865
STEP 76866
STEP 76867
STEP 76868
STEP 76869
STEP 76870
STEP 76871
STEP 76872
STEP 76873
STEP 76874
STEP 76875
STEP 76876
STEP 76877
STEP 76878
STEP 76879
STEP 76880
STEP 76881
STEP 76882
STEP 76883
STEP 76884
STEP 76885
STEP 76886
STEP 76887
STEP 76888
STEP 76889
STEP 76890
STEP 76891
STEP 76892
STEP 76893
STEP 76894
STEP 76895
STEP 76896
STEP 76897
STEP 76898
STEP 76899
STEP 76900
STEP 76901
STEP 76902
STEP 76903
STEP 76904
STEP 76905
STEP 76906
STEP 76907
STEP 76908
STEP 76909
STEP 76910
STEP 76911
STEP 76912

STEP 77870
STEP 77871
STEP 77872
STEP 77873
STEP 77874
STEP 77875
STEP 77876
STEP 77877
STEP 77878
STEP 77879
STEP 77880
STEP 77881
STEP 77882
STEP 77883
STEP 77884
STEP 77885
STEP 77886
STEP 77887
STEP 77888
STEP 77889
STEP 77890
STEP 77891
STEP 77892
STEP 77893
STEP 77894
STEP 77895
STEP 77896
STEP 77897
STEP 77898
STEP 77899
STEP 77900
STEP 77901
STEP 77902
STEP 77903
STEP 77904
STEP 77905
STEP 77906
STEP 77907
STEP 77908
STEP 77909
STEP 77910
STEP 77911
STEP 77912
STEP 77913
STEP 77914
STEP 77915
STEP 77916
STEP 77917
STEP 77918
STEP 77919
STEP 77920
STEP 77921
STEP 77922
STEP 77923
STEP 77924
STEP 77925
STEP 77926
STEP 77927
STEP 77928
STEP 77929
STEP 77930
STEP 77931
STEP 77932
STEP 77933
STEP 77934
STEP 77935
STEP 77936
STEP 77937
STEP 77938
STEP 77939
STEP 77940
STEP 77941
STEP 77942
STEP 77943
STEP 77944
STEP 77945
STEP 77946
STEP 77947
STEP 77948
STEP 77949
STEP 77950
STEP 77951
STEP 77952
STEP 77953
STEP 77954
STEP 77955
STEP 77956
STEP 77957
STEP 77958
STEP 77959
STEP 77960

STEP 78869
STEP 78870
STEP 78871
STEP 78872
STEP 78873
STEP 78874
STEP 78875
STEP 78876
STEP 78877
STEP 78878
STEP 78879
STEP 78880
STEP 78881
STEP 78882
STEP 78883
STEP 78884
STEP 78885
STEP 78886
STEP 78887
STEP 78888
STEP 78889
STEP 78890
STEP 78891
STEP 78892
STEP 78893
STEP 78894
STEP 78895
STEP 78896
STEP 78897
STEP 78898
STEP 78899
STEP 78900
STEP 78901
STEP 78902
STEP 78903
STEP 78904
STEP 78905
STEP 78906
STEP 78907
STEP 78908
STEP 78909
STEP 78910
STEP 78911
STEP 78912
STEP 78913
STEP 78914
STEP 78915
STEP 78916
STEP 78917
STEP 78918
STEP 78919
STEP 78920
STEP 78921
STEP 78922
STEP 78923
STEP 78924
STEP 78925
STEP 78926
STEP 78927
STEP 78928
STEP 78929
STEP 78930
STEP 78931
STEP 78932
STEP 78933
STEP 78934
STEP 78935
STEP 78936
STEP 78937
STEP 78938
STEP 78939
STEP 78940
STEP 78941
STEP 78942
STEP 78943
STEP 78944
STEP 78945
STEP 78946
STEP 78947
STEP 78948
STEP 78949
STEP 78950
STEP 78951
STEP 78952
STEP 78953
STEP 78954
STEP 78955
STEP 78956
STEP 78957
STEP 78958
STEP 78959

STEP 79912
STEP 79913
STEP 79914
STEP 79915
STEP 79916
STEP 79917
STEP 79918
STEP 79919
STEP 79920
STEP 79921
STEP 79922
STEP 79923
STEP 79924
STEP 79925
STEP 79926
STEP 79927
STEP 79928
STEP 79929
STEP 79930
STEP 79931
STEP 79932
STEP 79933
STEP 79934
STEP 79935
STEP 79936
STEP 79937
STEP 79938
STEP 79939
STEP 79940
STEP 79941
STEP 79942
STEP 79943
STEP 79944
STEP 79945
STEP 79946
STEP 79947
STEP 79948
STEP 79949
STEP 79950
STEP 79951
STEP 79952
STEP 79953
STEP 79954
STEP 79955
STEP 79956
STEP 79957
STEP 79958
STEP 79959
STEP 79960
STEP 79961
STEP 79962
STEP 79963
STEP 79964
STEP 79965
STEP 79966
STEP 79967
STEP 79968
STEP 79969
STEP 79970
STEP 79971
STEP 79972
STEP 79973
STEP 79974
STEP 79975
STEP 79976
STEP 79977
STEP 79978
STEP 79979
STEP 79980
STEP 79981
STEP 79982
STEP 79983
STEP 79984
STEP 79985
STEP 79986
STEP 79987
STEP 79988
STEP 79989
STEP 79990
STEP 79991
STEP 79992
STEP 79993
STEP 79994
STEP 79995
STEP 79996
STEP 79997
STEP 79998
STEP 79999


In [12]:
# MAKE OUTPUT.CSV
step = 0
row = []

for data,ratio in zip(data_list, true_ratios):
    step = data.step
    r_embb = data.r_embb   
    r_urllc = data.r_urllc 
    r_miot = data.r_miot 
    r_mmtc = data.r_mmtc  
    r_voice = data.r_voice   
    t_embb = ratio.true_embb
    t_urllc = ratio.true_urllc
    t_miot = ratio.true_miot
    t_mmtc = ratio.true_mmtc
    t_voice = ratio.true_voice
    t_band = ratio.band
    
    data = MakeCsv(step, r_embb, t_embb, r_urllc, t_urllc, r_miot, t_miot, r_mmtc, t_mmtc, r_voice, t_voice, t_band) 
    row.append(data)
    
with open('output.csv', mode='w') as csv_file:
    fieldnames = ['round', 'embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true', 'br']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in row:
        writer.writerow({'round':item.step, 'embb_req':item.r_embb, 'embb_true':item.t_embb, 'urllc_req':item.r_urllc, 'urllc_true':item.t_urllc,'miot_req':item.r_miot, 'miot_true':item.t_miot,'mmtc_req':item.r_mmtc, 'mmtc_true':item.t_mmtc,'voice_req':item.r_voice, 'voice_true':item.t_voice, 'br':item.band})
